# The barometer

A notebook to aid in the meausurment of atomospheric pressure as required in the Rüchardt experiment in the part II year labs at UTAS

Written by A. J. McCulloch, May 2021

## Import packages

To streamline operations in Python, packages can be imported to perform a host of various tasks.
* _numpy_ is a fundamental package for numerical and mathematical computing
* _pandas_ is a data analysis tool
* _matplotlib_ is used for visualisations
* _glob_ is used for pathnames
* _scipy_ is used for high-level scientific computing

In [8]:
import numpy as np # Numpy for all things numeric
import pandas as pd # Pandas for storing and manipulating data
import requests
from datetime import datetime
from uncertainties import ufloat

import matplotlib.pyplot as plt # matplotlib for plotting
import glob # glob required for searching directories

from scipy.interpolate import interp1d # Required for interpolation of the calibration curves
from scipy.stats import linregress # Required for linear regression
from scipy import integrate # Required for integration
from matplotlib import cm # Required for colour mapping

import os, sys


# Below is some magic to make things work better, don't wory about this
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## Corrections to the measured pressure

### Temperature corrections based on tabulated data

Put in the measurement values and unvertainties, noting that these are in the units of the barometer

In [9]:
Height = 30.42 # Barometer reading [inches of Hg]
Height_u = 0.05 # Measurement uncertainty of barometer reading
Temperature = 57  # Temperature reading [F]
Temperature_u = 0.5 # Measurement uncertainty of temperature reading

A function which returns the correction factor needed to compensate for the effect of temperature on the pressure reading. Inputs are _h_ which is the pressure (height measurement in inches) and _T_ the temperature in degrees fahrenheit, and the function returns the correction factor and the corrected temperature.

In [10]:
def correction(h, T):
    
    # This function takes the pressure reading and the temperatue and applies a correction based on tabulated values from the manufacturer
    # Currently, input values are rounded to the closest tabulated values
    # The processed could be imporved by interpolating the tabulated values
    
    df_c = pd.read_csv('Data/Corrections.csv') # Import the tabulated correction data
    readings = [] # Initialise an array for the pressure readings in the data
    # Extract the pressure readings from the reference data
    for i in df_c.columns.values:
        try:
            readings.append(float(i)) # The first column in a temperature heading (a string) and thus will fail float()
        except:
            pass
    
    # Very basic error handling: check the input values to the function are in the accepted data range
    if (min(readings) < h < max(readings)) & (df_c['Temperature [F]'].min() < T < df_c['Temperature [F]'].max()):
        try:
            # Find the tabluated value with the temperature rounded to the nearest degree, and the pressure rounded to the nearest half degree
            corr = df_c[df_c['Temperature [F]'] == round(T)][str(round(h * 2)/2)].values[0]
            return corr, h-corr
        except TypeError:
            # Find the tabluated value with the temperature rounded to the nearest degree, and the pressure rounded to the nearest half degree
            corr = df_c[df_c['Temperature [F]'] == round(T.n)][str(round(h.n * 2)/2)].values[0]
            return corr, h-corr
        except:
            # This error should not really be thrown: if the values passed the verification above, it should work!
            raise ValueError('Incorrect data format, check values manually')
    else:
        if len(readings) == 0:
            # The columns (pressure readings) were not found, likely because the file wasn't there or corrupted
            raise ValueError('Verify the location/fidelity of the corrections file')
        elif min(readings) < h < max(readings):
            # If the pressure reading is good, the temperature must be bad
            raise ValueError('Temperature out of range')
        elif df_c['Temperature [F]'].min() < T < df_c['Temperature [F]'].max():
            # If the temperature reading is good, the pressure must be bad
            raise ValueError('Pressure out of range')
        else:
            # All values are bad!
            raise ValueError('Temperature and pressure out of range')

A function to covert the pressure from inches of mercury to Pascals

In [11]:
def pressure_convert(p):
    
    # Function to convert inches of mercury to Pascals
    # conversion: 1 inch = 25.4 mm, 1 mm Hg = 133.322387415 Pa
    
    return p * 25.4 * 133.322387415

The benefits of using python are its versitiliy and broad applicability. At this point, we are measing the ambient pressure using a barometer, but the pressure is actively monitored as part of weather monitoring. The Bureau of Meterology (BoM) has a Hobart based weather station and publishes the measurements in real time, which we can collect and check that our result is consistent. To do this, we make use of the _requests_ library, which can collect the data from the BoM. We explicitly ask for the data in JavaScript Object Notation (JSON) format, which is routinely used to store web-based data. We can use the _requests.json()_ functionality to interpret the data, and with a bit of simple manipulation return the pressure as measured in Hobart!

In [12]:
def get_pressure(ret=False):
    BOM = 'http://www.bom.gov.au/fwo/IDT60901/IDT60901.94970.json' # Hobart weather station
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0'}
    r = requests.get(BOM, headers = headers)

    data =  r.json()['observations']['data'][0]

    BOM_data = []
    for f in ['local_date_time_full', 'press']:
        if f == 'local_date_time_full':
            BOM_data.append(datetime.strptime(data[f], format('%Y%m%d%H%M%S')))
        else:
            BOM_data.append(data[f])

    print(f"Pressure measured at {BOM_data[0].strftime('%H%M on %d/%m/%y')} by the BOM: {BOM_data[1]} hPa")
    
    if ret == True:
        return BOM_data[1]

In [13]:
def return_hPa(Height, Height_u, Temperature, Temperature_u):
    h_meas = ufloat(Height, Height_u)
    T_meas = ufloat(Temperature, Temperature_u)
    
    p_hPa = pressure_convert(correction(h_meas, T_meas)[1])/1e2
    print(f'Input values return pressure of {p_hPa} hPa')
    
    get_pressure()
    
    return p_hPa

In [14]:
p = return_hPa(Height, Height_u, Temperature, Temperature_u)

Input values return pressure of 1027.5+/-1.7 hPa
Pressure measured at 1330 on 06/04/22 by the BOM: 1025.9 hPa


### Corrections based of formulae

A more complete correction of the pressure requires three separate steps: a correction of thermal effects, a correction of the local grivational potential, and a correction due to the altitude of the measurement. We begin by defining the functions as sourced from [this barometer manual](https://novalynx.com/manuals/230-7410-7420-manual.pdf):

In [15]:
def corr_T(P,T):
    L = 1.02e-5 
    M = 10.10e-5
    return P * (((1+L*(T-62))/(1+M*(T-32)))-1)

def corr_g(P, phi):
    return P * ((980.616/980.665) * (1 - (2637.3 * np.cos(2*phi) + 5.9 * (np.cos(2*phi))**2)/1e6) - 1)

def corr_s(H):
    P0 = 29.921
    return P0 * (1 - (1 - (6.5e-3/288.16)*H) ** 5.2561)

To reproduce table 5.1.A from the barometer manual, we can definte a function to print each correction and pressure:

In [16]:
def pressure_corrections(P, T, phi, H, SLP = False):
    C = corr_T(P, T)
    print(f"The temperature correction is {C:.3f} inches Hg")
    P += C
    print(f"The temperature corrected pressure is {P:.3f} inches Hg\n")
    C = corr_g(P, np.radians(phi_ex))
    
    print(f"The gravity correction is {C:.3f} inches Hg")
    P += C
    print(f"The local (gravity corrected) pressure is {P:.3f} inches Hg\n")
    C = corr_g(P, np.radians(phi_ex))
    
    if SLP == True:
        C = corr_s(H)
        print(f"The sea-level differential is {C:.3f} inches Hg")
        P += C    
        print(f"The equivalent sea-level pressure is {P:.3f} inches Hg")

    return P

and then use the example values as provided to ensure consistency

In [17]:
P_ex, T_ex, phi_ex, H_ex = (29.5, 68, 40, 152.4)

example_pressure = pressure_corrections(P_ex, T_ex, phi_ex, H_ex, True)

The temperature correction is -0.105 inches Hg
The temperature corrected pressure is 29.395 inches Hg

The gravity correction is -0.015 inches Hg
The local (gravity corrected) pressure is 29.380 inches Hg

The sea-level differential is 0.537 inches Hg
The equivalent sea-level pressure is 29.917 inches Hg


For our data, we define another function:

In [18]:
def full_corrections_hPa(Height, Height_u, Temperature, Temperature_u, phi = 42.9, H = 31.54):
    h_meas = ufloat(Height, Height_u)
    T_meas = ufloat(Temperature, Temperature_u)
    
    with HiddenPrints():
        p_hPa = pressure_convert(pressure_corrections(h_meas, T_meas, phi, H))/1e2
        
    print(f'Input values return pressure of {p_hPa} hPa')
    
    return p_hPa

into which we can put the measured values (as per the section above):

In [19]:
inputvals = (Height, Height_u, Temperature, Temperature_u)
P = full_corrections_hPa(*inputvals)

Input values return pressure of 1027.0+/-1.7 hPa


From [the weather station description page](http://www.bom.gov.au/products/IDT60901/IDT60901.94970.shtml), the altitude of the station is given as 50.5 m, meaning we can correct this a common height and check for consistency

In [20]:
def localvsBOM(Height, Height_u, Temperature, Temperature_u, phi = 42.9, H = 31.54):
    h_meas = ufloat(Height, Height_u)
    T_meas = ufloat(Temperature, Temperature_u)
    
    with HiddenPrints():
        p_hPa = pressure_convert(pressure_corrections(h_meas, T_meas, phi, H, True))/1e2
        P_BOM = get_pressure(ret=True)
    P_BOM += pressure_convert(corr_s(50.5))/1e2
    
    d = abs((P_BOM - p_hPa.n)/P_BOM) * 100
    
    print(f"The pressure corrected for sea level is {p_hPa} hPa based on the input values and {P_BOM:.1f} hPa as measured by the BOM\nThe discrepancy between these values is {d:.1f}%")

In [21]:
localvsBOM(*inputvals)

The pressure corrected for sea level is 1030.8+/-1.7 hPa based on the input values and 1032.0 hPa as measured by the BOM
The discrepancy between these values is 0.1%
